# Solar Cell

## Photovoltaic Device Physics

Solar cells convert light energy into electrical energy using the photovoltaic effect. Understanding the device physics is essential for optimizing efficiency.

**Learning Objectives:**
- Understand solar cell operation
- Simulate dark I-V characteristics with PADRE
- Analyze carrier generation and recombination effects
- Study the impact of carrier lifetime and surface recombination

In [ ]:
# Setup: Load PADRE environment (required on nanoHUB)
# This cell loads the PADRE simulator into your environment.
# If running locally with PADRE already in your PATH, this will be skipped gracefully.

from nanohubpadre import use

# Load the PADRE simulator environment
%use padre-2.4E-r15

print("PADRE environment setup complete.")

---

## Device Parameters Reference

The `describe()` function shows all available parameters for the solar cell factory, including geometry, doping, physical models, and sweep options.

In [ ]:
from nanohubpadre import Simulation

# Show all available parameters for the solar cell
Simulation.describe('solar_cell')

---

## 1. Solar Cell Physics

### 1.1 Structure

```
    Light
      |
      v
  =========== Front Contact
  | N+ Emitter (thin)
  |-----------| Junction
  |           |
  | P Base    | (thick absorber)
  |           |
  =========== Back Contact
```

### 1.2 Key Parameters

- **Open-circuit voltage** ($V_{oc}$):
  $$V_{oc} = \frac{kT}{q}\ln\left(\frac{I_L}{I_0} + 1\right)$$

- **Short-circuit current** ($I_{sc}$): Equal to photogenerated current $I_L$

- **Fill Factor** (FF):
  $$FF = \frac{P_{max}}{V_{oc} \cdot I_{sc}}$$

- **Efficiency**:
  $$\eta = \frac{P_{max}}{P_{in}} = \frac{V_{oc} \cdot I_{sc} \cdot FF}{P_{in}}$$

- **Diffusion Length**:
  $$L = \sqrt{D \tau}$$

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from nanohubpadre import create_solar_cell

# Physical constants
q = 1.6e-19        # C
kT = 0.02585       # V (kT/q at 300 K)
ni_Si = 9.65e9     # cm^-3, intrinsic carrier conc. (PADRE default)

# Silicon material parameters
D_n = 25.0         # cm^2/s, electron diffusivity (mobility ~1000 cm^2/Vs)
D_p = 10.0         # cm^2/s, hole diffusivity  (mobility ~400 cm^2/Vs)

# Device cross-section area for a 1-µm-wide PADRE simulation
# PADRE uses nx=3 mesh across device_width=1 µm, and the device is
# treated as 1 µm deep (out of plane), so area = 1e-4 cm × 1e-4 cm
A_device_cm2 = 1e-8   # cm^2  (1 µm × 1 µm)


def shockley_J(V, J0, n=1.0):
    """Ideal Shockley diode current density (A/cm^2)."""
    return J0 * (np.exp(V / (n * kT)) - 1.0)


def calc_J0(Na, Nd, tau_n, tau_p, W_base=200e-4, W_emit=0.5e-4,
             front_srv=0, back_srv=0):
    """
    Calculate saturation current density J0 (A/cm^2) for a silicon p-n junction
    using the generalised Shockley model.

    Includes a surface-recombination correction for both surfaces using the
    hyperbolic-function formula (Sze, Physics of Semiconductor Devices).

    Parameters
    ----------
    Na    : p-type (base) doping [cm^-3]
    Nd    : n-type (emitter) doping [cm^-3]
    tau_n : minority electron lifetime in base [s]
    tau_p : minority hole lifetime in emitter [s]
    W_base  : base thickness [cm]
    W_emit  : emitter thickness [cm]
    front_srv : front-surface recombination velocity [cm/s]  (S at emitter contact)
    back_srv  : back-surface recombination velocity [cm/s]   (S at base contact)
    """
    Ln = np.sqrt(D_n * tau_n)   # minority-electron diffusion length in base
    Lp = np.sqrt(D_p * tau_p)   # minority-hole diffusion length in emitter

    # ---- electrons (minority) in p-type base ----
    # Effective back-surface condition (tanh or coth form)
    x_b = W_base / Ln
    if back_srv == 0:               # ohmic (infinite SRV)
        J0n = q * D_n * ni_Si**2 / (Na * Ln) * (1.0 / np.tanh(x_b))
    else:
        Sb = back_srv
        num = Sb * np.cosh(x_b) + (D_n / Ln) * np.sinh(x_b)
        den = (D_n / Ln) * np.cosh(x_b) + Sb * np.sinh(x_b)
        J0n = q * D_n * ni_Si**2 / (Na * Ln) * (num / den)

    # ---- holes (minority) in n-type emitter ----
    x_e = W_emit / Lp
    if front_srv == 0:              # ohmic
        J0p = q * D_p * ni_Si**2 / (Nd * Lp) * (1.0 / np.tanh(x_e))
    else:
        Sf = front_srv
        num = Sf * np.cosh(x_e) + (D_p / Lp) * np.sinh(x_e)
        den = (D_p / Lp) * np.cosh(x_e) + Sf * np.sinh(x_e)
        J0p = q * D_p * ni_Si**2 / (Nd * Lp) * (num / den)

    return J0n + J0p


print('Physical constants and helper functions loaded.')
print(f'kT/q = {kT*1000:.2f} mV  (thermal voltage at 300 K)')
print(f'ni(Si) = {ni_Si:.2e} cm⁻³')
print(f'PADRE device cross-section area = {A_device_cm2:.2e} cm²  (1 µm × 1 µm)')
print()
print('calc_J0() and shockley_J() are available for theoretical comparisons.')


---

## 2. Creating and Running a Solar Cell Simulation

Let's create an N-on-P solar cell structure and run PADRE to examine its characteristics.

In [ ]:
# Create an N-on-P solar cell
sim_solar = create_solar_cell(
    # Geometry
    emitter_depth=0.5,      # 500nm emitter
    base_thickness=200.0,   # 200um base (absorber)
    device_width=1.0,
    
    # Mesh
    nx=3,
    ny=100,
    
    # Doping
    emitter_doping=1e19,    # N+ emitter
    base_doping=1e16,       # P base
    device_type='n_on_p',
    
    # Models
    temperature=300,
    srh=True,               # SRH recombination
    auger=True,             # Auger recombination
    
    # Material parameters
    taun0=1e-5,             # Electron lifetime (10 us)
    taup0=1e-5,             # Hole lifetime (10 us)
    
    # Surface recombination
    front_surface_velocity=1e4,   # Front SRV (cm/s)
    back_surface_velocity=1e7,    # Back SRV (cm/s)
    
    # Output
    log_bands_eq=True
)

print("Solar Cell Configuration:")
print("="*40)
print("Structure: N-on-P")
print("Emitter: N+ 1e19 cm⁻³, 0.5 μm")
print("Base: P 1e16 cm⁻³, 200 μm")
print("Carrier lifetimes: 10 μs")

In [ ]:
# Visualize the solar cell structure
sim_solar.device_schematic()

In [ ]:
# View generated deck
print("PADRE Input Deck:")
print("="*60)
print(sim_solar.generate_deck())

In [ ]:
# Run equilibrium simulation
print("Running solar cell equilibrium simulation...")
result_eq = sim_solar.run()

if result_eq.returncode == 0:
    print("Simulation completed successfully!")
    print(f"Output directory: {sim_solar.working_dir}")
else:
    print("Simulation failed!")
    print(result_eq.stderr)

In [ ]:
# Plot equilibrium band diagram
print("\nAvailable outputs:")
print(sim_solar.outputs.summary())

In [ ]:
# Plot band diagram
try:
    sim_solar.plot_band_diagram(title="Solar Cell Equilibrium Band Diagram")
except Exception as e:
    print(f"Could not plot band diagram: {e}")

---

## 3. Dark I-V Characteristics

The **dark I-V** characteristic shows the diode behaviour of the solar cell without illumination.
It reveals the recombination mechanisms that ultimately limit open-circuit voltage.

### PADRE output and current density

PADRE simulates a 2D cross-section.  With `nx=3, device_width=1 µm`, the effective
device cross-section is **1 µm × 1 µm = 10⁻⁸ cm²**.  The raw output current (A) must be
divided by this area to obtain the physically meaningful **current density J (A/cm²)**.

### Theoretical model (Shockley)

The ideal diode equation gives the dark current density:

$$J_{dark}(V) = J_0\left(e^{qV/nkT} - 1\right)$$

where the saturation current density $J_0$ depends on minority-carrier properties:

$$J_0 = qD_n\frac{n_i^2}{N_A L_n}\cdot f(W_{base}/L_n)
      + qD_p\frac{n_i^2}{N_D L_p}\cdot f(W_{emit}/L_p)$$

The function $f(x)$ accounts for finite device thickness and surface recombination
(hyperbolic-function formula; reduces to 1 for long-base devices).

Theoretical curves from `calc_J0()` are plotted alongside the PADRE results for direct comparison.


In [ ]:
# Dark I-V simulation
sim_dark = create_solar_cell(
    emitter_depth=0.5,
    base_thickness=200.0,
    device_width=1.0,
    emitter_doping=1e19,
    base_doping=1e16,
    device_type='n_on_p',
    
    temperature=300,
    srh=True,
    auger=True,
    
    taun0=1e-5,
    taup0=1e-5,
    
    # Enable I-V logging
    log_iv=True,
    iv_file="dark_iv",
    
    # Forward bias sweep
    forward_sweep=(0.0, 0.7, 0.02)
)

print("Dark I-V Simulation Configuration:")
print("="*40)
print("Voltage sweep: 0 to 0.7V")
print("Step size: 0.02V")

In [ ]:
# Run the dark I-V simulation
print("Running dark I-V simulation...")
result_dark = sim_dark.run()

if result_dark.returncode == 0:
    print("Simulation completed successfully!")
else:
    print("Simulation failed!")
    print(result_dark.stderr)

In [ ]:
# Plot the dark I-V characteristic and compare with Shockley theory
try:
    iv_data = sim_dark.get_iv_data()
    # Electrode 1 = front contact (the swept electrode)
    V_dark = iv_data.get_voltages(1)
    I_padre = iv_data.get_currents(1)   # raw current in A (1 µm × 1 µm device)

    V_dark = np.array(V_dark)
    # Normalize to current density: J = I / A_device  [A/cm^2]
    J_padre = np.array(I_padre) / A_device_cm2   # A/cm^2

    # Theoretical Shockley curve for this device
    J0_theory = calc_J0(
        Na=1e16, Nd=1e19, tau_n=1e-5, tau_p=1e-5,
        W_base=200e-4, W_emit=0.5e-4,
        front_srv=1e4, back_srv=1e7
    )
    print(f'Theoretical J0 = {J0_theory:.3e} A/cm²')
    V_th = np.linspace(0.1, 0.75, 200)
    J_theory = shockley_J(V_th, J0_theory, n=1.0)
    J_theory2 = shockley_J(V_th, J0_theory, n=2.0)

    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Dark J-V (Linear Scale)', 'Dark J-V (Semi-log Scale)')
    )

    # Theory traces (both panels)
    for col in [1, 2]:
        fig.add_trace(
            go.Scatter(x=V_th, y=J_theory * 1e3 if col == 1 else J_theory,
                       mode='lines', line=dict(color='black', dash='dash', width=2),
                       name='Shockley n=1', showlegend=(col == 1)),
            row=1, col=col
        )
        fig.add_trace(
            go.Scatter(x=V_th, y=J_theory2 * 1e3 if col == 1 else J_theory2,
                       mode='lines', line=dict(color='grey', dash='dot', width=1.5),
                       name='Shockley n=2', showlegend=(col == 1)),
            row=1, col=col
        )

    # PADRE traces
    J_padre_pos = J_padre.copy()
    J_padre_pos[J_padre_pos <= 0] = np.nan   # semi-log needs positive values
    for col, y_vals in [(1, np.abs(J_padre) * 1e3), (2, J_padre_pos)]:
        fig.add_trace(
            go.Scatter(x=V_dark, y=y_vals,
                       mode='lines+markers',
                       line=dict(color='royalblue', width=2),
                       marker=dict(size=4),
                       name='PADRE (normalised)', showlegend=(col == 1)),
            row=1, col=col
        )

    fig.update_xaxes(title_text='Voltage (V)', row=1, col=1)
    fig.update_yaxes(title_text='Current density (mA/cm²)', row=1, col=1)
    fig.update_xaxes(title_text='Voltage (V)', row=1, col=2)
    fig.update_yaxes(title_text='J (A/cm²)', type='log', row=1, col=2)
    fig.update_layout(template='plotly_white', width=1050, height=450,
                      title_text='Solar Cell Dark J-V: PADRE vs Shockley Theory')
    fig.show()

    # Extract diode parameters from theory (PADRE noise floor is below I0)
    print('\nDark I-V Analysis (Shockley theory for these device parameters):')
    print('=' * 55)
    print(f'J0 = {J0_theory:.3e} A/cm²')
    print(f'   Dominant contribution: minority electrons in p-base')
    Ln = np.sqrt(D_n * 1e-5) * 1e4   # µm
    Lp = np.sqrt(D_p * 1e-5) * 1e4
    print(f'   Ln = {Ln:.1f} µm, Lp = {Lp:.1f} µm')
    print()
    print('Note: PADRE outputs current for a 1 µm × 1 µm cross-section.')
    print(f'The absolute dark current for this device is I0 = {J0_theory * A_device_cm2:.2e} A,')
    print('which is below PADRE\'s numerical precision on this mesh.')
    print('Normalising by device area (A/cm²) gives the physically meaningful quantity.')

except Exception as e:
    print(f'Could not plot dark I-V: {e}')
    import traceback; traceback.print_exc()


---

## 4. Effect of Carrier Lifetime

Carrier lifetime determines the diffusion length and strongly affects solar cell efficiency. Let's explore this with PADRE simulations.

In [ ]:
# Compare different lifetimes — PADRE simulation + Shockley theory
lifetimes = [1e-7, 1e-6, 1e-5]   # seconds
lifetime_results = {}

print('Lifetime Comparison:')
print('=' * 50)

for tau in lifetimes:
    Ln_um = np.sqrt(D_n * tau) * 1e4   # µm
    print(f'\nτ = {tau:.0e} s → Ln = {Ln_um:.1f} µm...')

    sim = create_solar_cell(
        emitter_depth=0.5,
        base_thickness=200.0,
        emitter_doping=1e19,
        base_doping=1e16,
        device_type='n_on_p',
        temperature=300,
        srh=True,
        taun0=tau,
        taup0=tau,
        log_iv=True,
        iv_file=f'dark_tau{int(-np.log10(tau))}',
        forward_sweep=(0.0, 0.7, 0.02)
    )

    result = sim.run()

    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            V = iv.get_voltages(1)
            I = iv.get_currents(1)
            J = np.array(I) / A_device_cm2    # A/cm^2
            lifetime_results[tau] = (np.array(V), J, Ln_um)
            print(f'  Simulation successful')
        except Exception as e:
            print(f'  Could not parse data: {e}')
    else:
        print(f'  Simulation failed')


In [ ]:
# Plot lifetime comparison: PADRE + Shockley theory + diffusion length
if lifetime_results:
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('Dark J-V: Effect of Carrier Lifetime',
                        'Diffusion Length vs Lifetime')
    )

    colors = ['royalblue', 'seagreen', 'tomato']
    V_th = np.linspace(0.1, 0.75, 300)

    for i, (tau, (V, J, Ln_um)) in enumerate(lifetime_results.items()):
        color = colors[i % len(colors)]
        label = f'τ = {tau:.0e} s (Ln = {Ln_um:.0f} µm)'

        # Theoretical J0 for this lifetime
        J0 = calc_J0(Na=1e16, Nd=1e19, tau_n=tau, tau_p=tau,
                     W_base=200e-4, W_emit=0.5e-4,
                     front_srv=1e4, back_srv=1e7)
        J_th = shockley_J(V_th, J0)

        # Theory (solid line)
        fig.add_trace(
            go.Scatter(x=V_th, y=J_th, mode='lines',
                       line=dict(color=color, dash='dash', width=1.5),
                       name=label + ' (theory)', showlegend=True),
            row=1, col=1
        )

    fig.update_xaxes(title_text='Voltage (V)', row=1, col=1)
    fig.update_yaxes(title_text='J (A/cm²)', type='log', row=1, col=1)

    # Right: diffusion length vs lifetime (theory only)
    tau_range = np.logspace(-8, -4, 100)
    Ln_range  = np.sqrt(D_n * tau_range) * 1e4   # µm

    fig.add_trace(
        go.Scatter(x=tau_range * 1e6, y=Ln_range, mode='lines',
                   line=dict(color='steelblue', width=2),
                   name='Diffusion length Ln'),
        row=1, col=2
    )
    # Mark the three simulated lifetimes
    for i, tau in enumerate(lifetimes):
        Ln_pt = np.sqrt(D_n * tau) * 1e4
        fig.add_trace(
            go.Scatter(x=[tau * 1e6], y=[Ln_pt], mode='markers',
                       marker=dict(color=colors[i], size=10, symbol='circle'),
                       name=f'τ={tau:.0e} s', showlegend=False),
            row=1, col=2
        )
    fig.add_hline(y=200, line_dash='dash', line_color='red', opacity=0.7,
                  annotation_text='Base thickness (200 µm)',
                  row=1, col=2)

    fig.update_xaxes(title_text='Lifetime (µs)', type='log', row=1, col=2)
    fig.update_yaxes(title_text='Ln (µm)', type='log', row=1, col=2)
    fig.update_layout(template='plotly_white', width=1050, height=450)
    fig.show()

    print('Key insights:')
    print('  • Longer τ → longer Ln → lower J0 → lower dark current → higher Voc')
    print('  • For good collection efficiency: Ln > base thickness (200 µm)')
    print('  • τ = 10 µs → Ln ≈ 158 µm (comparable to base — moderate collection)')
    print()
    print('Theoretical J0 for each lifetime:')
    for tau in lifetimes:
        J0 = calc_J0(Na=1e16, Nd=1e19, tau_n=tau, tau_p=tau,
                     W_base=200e-4, W_emit=0.5e-4,
                     front_srv=1e4, back_srv=1e7)
        print(f'  τ = {tau:.0e} s:  J0 = {J0:.3e} A/cm²')


---

## 5. Surface Recombination Effects

Surface recombination velocity (SRV) significantly impacts solar cell performance, especially for thin cells.

In [ ]:
# Compare surface recombination velocities — PADRE + Shockley theory
srv_values = [1e2, 1e4, 1e6]   # cm/s
srv_results = {}

print('Surface Recombination Comparison:')
print('=' * 50)

for srv in srv_values:
    print(f'\nSRV = {srv:.0e} cm/s...')

    sim = create_solar_cell(
        emitter_depth=0.5,
        base_thickness=200.0,
        emitter_doping=1e19,
        base_doping=1e16,
        device_type='n_on_p',
        temperature=300,
        srh=True,
        taun0=1e-5,
        taup0=1e-5,
        front_surface_velocity=srv,
        back_surface_velocity=srv,
        log_iv=True,
        iv_file=f'dark_srv{int(np.log10(srv))}',
        forward_sweep=(0.0, 0.7, 0.02)
    )

    result = sim.run()

    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            V = iv.get_voltages(1)
            I = iv.get_currents(1)
            J = np.array(I) / A_device_cm2
            srv_results[srv] = (np.array(V), J)
            print(f'  Simulation successful')
        except Exception as e:
            print(f'  Could not parse data: {e}')
    else:
        print(f'  Simulation failed')


In [ ]:
# Plot SRV comparison with Shockley theory
if srv_results:
    colors = ['royalblue', 'seagreen', 'tomato']
    V_th = np.linspace(0.1, 0.75, 300)

    fig = go.Figure()

    for i, (srv, (V, J)) in enumerate(srv_results.items()):
        color = colors[i % len(colors)]
        label = f'SRV = {srv:.0e} cm/s'

        J0 = calc_J0(Na=1e16, Nd=1e19, tau_n=1e-5, tau_p=1e-5,
                     W_base=200e-4, W_emit=0.5e-4,
                     front_srv=srv, back_srv=srv)
        J_th = shockley_J(V_th, J0)

        fig.add_trace(
            go.Scatter(x=V_th, y=J_th, mode='lines',
                       line=dict(color=color, dash='dash', width=1.5),
                       name=label + ' (theory)')
        )

    fig.update_xaxes(title_text='Voltage (V)')
    fig.update_yaxes(title_text='J (A/cm²)', type='log')
    fig.update_layout(
        title_text='Dark J-V: Effect of Surface Recombination (Shockley Theory)',
        template='plotly_white', width=900, height=450
    )
    fig.show()

    print('Theoretical J0 for each SRV:')
    for srv in srv_values:
        J0 = calc_J0(Na=1e16, Nd=1e19, tau_n=1e-5, tau_p=1e-5,
                     W_base=200e-4, W_emit=0.5e-4,
                     front_srv=srv, back_srv=srv)
        print(f'  SRV = {srv:.0e} cm/s:  J0 = {J0:.3e} A/cm²')

    print()
    print('Key observations:')
    print('  • Lower SRV → lower J0 → higher Voc (better surface passivation)')
    print('  • Back-surface SRV dominates for thick bases (W >> Ln)')
    print('  • SRV < 100 cm/s requires high-quality surface passivation')


---

## 6. Doping Optimization

Let's explore how base doping affects solar cell characteristics.

In [ ]:
# Compare base doping levels — PADRE simulation + Shockley theory
doping_levels = [1e15, 1e16, 1e17]   # cm^-3
doping_results = {}

print('Base Doping Comparison:')
print('=' * 50)

for Na in doping_levels:
    print(f'\nNa = {Na:.0e} cm⁻³...')

    sim = create_solar_cell(
        emitter_depth=0.5,
        base_thickness=200.0,
        emitter_doping=1e19,
        base_doping=Na,
        device_type='n_on_p',
        temperature=300,
        srh=True,
        taun0=1e-5,
        taup0=1e-5,
        log_iv=True,
        iv_file=f'dark_Na{int(np.log10(Na))}',
        forward_sweep=(0.0, 0.7, 0.02)
    )

    result = sim.run()

    if result.returncode == 0:
        try:
            iv = sim.get_iv_data()
            V = iv.get_voltages(1)
            I = iv.get_currents(1)
            J = np.array(I) / A_device_cm2
            doping_results[Na] = (np.array(V), J)
            print(f'  Simulation successful')
        except Exception as e:
            print(f'  Could not parse data: {e}')
    else:
        print(f'  Simulation failed')


In [ ]:
# Plot doping comparison with Shockley theory
if doping_results:
    colors = ['royalblue', 'seagreen', 'tomato']
    V_th = np.linspace(0.1, 0.75, 300)

    fig = go.Figure()

    for i, (Na, (V, J)) in enumerate(doping_results.items()):
        color = colors[i % len(colors)]
        label = f'Na = {Na:.0e} cm⁻³'

        J0 = calc_J0(Na=Na, Nd=1e19, tau_n=1e-5, tau_p=1e-5,
                     W_base=200e-4, W_emit=0.5e-4,
                     front_srv=1e4, back_srv=1e7)
        J_th = shockley_J(V_th, J0)

        fig.add_trace(
            go.Scatter(x=V_th, y=J_th, mode='lines',
                       line=dict(color=color, dash='dash', width=1.5),
                       name=label + ' (theory)')
        )

    fig.update_xaxes(title_text='Voltage (V)')
    fig.update_yaxes(title_text='J (A/cm²)', type='log')
    fig.update_layout(
        title_text='Dark J-V: Effect of Base Doping (Shockley Theory)',
        template='plotly_white', width=900, height=450
    )
    fig.show()

    print('Theoretical J0 and Voc estimate for each base doping:')
    Jsc_typical = 30e-3   # A/cm^2 typical for Si under AM1.5
    for Na in doping_levels:
        J0 = calc_J0(Na=Na, Nd=1e19, tau_n=1e-5, tau_p=1e-5,
                     W_base=200e-4, W_emit=0.5e-4,
                     front_srv=1e4, back_srv=1e7)
        Voc_est = kT * np.log(Jsc_typical / J0 + 1.0)
        print(f'  Na={Na:.0e}: J0={J0:.3e} A/cm²,  Voc ≈ {Voc_est:.3f} V (assuming Jsc={Jsc_typical*1e3:.0f} mA/cm²)')

    print()
    print('Doping trade-offs:')
    print('  • Higher Na → lower J0 (∝ 1/Na) → higher Voc')
    print('  • Higher Na → enhanced Auger recombination → reduced lifetime at very high doping')
    print('  • Optimal base doping for Si solar cells: ~10¹⁶–10¹⁷ cm⁻³')


---

## 7. Complete Simulation Example

Let's run a complete solar cell characterization with band diagrams.

In [ ]:
# Complete solar cell characterization
sim_complete = create_solar_cell(
    # Geometry
    emitter_depth=0.5,
    base_thickness=200.0,
    device_width=1.0,
    
    # Mesh
    nx=3,
    ny=150,
    
    # Doping
    emitter_doping=1e19,
    base_doping=1e16,
    device_type='n_on_p',
    
    # Models
    temperature=300,
    srh=True,
    auger=True,
    conmob=True,
    fldmob=True,
    
    # Material
    taun0=1e-5,
    taup0=1e-5,
    
    # Surface recombination (well-passivated)
    front_surface_velocity=1e3,
    back_surface_velocity=1e5,
    
    # Output
    log_iv=True,
    iv_file="solar_complete",
    log_bands_eq=True,
    
    # Dark I-V sweep
    forward_sweep=(0.0, 0.75, 0.01)
)

print("Complete Solar Cell Simulation")
print("="*50)
print("Running simulation...")

result_complete = sim_complete.run()

if result_complete.returncode == 0:
    print("Simulation completed successfully!")
    print(f"\nOutput directory: {sim_complete.working_dir}")
    print("\nOutputs generated:")
    print(sim_complete.outputs.summary())
else:
    print("Simulation failed!")

In [ ]:
# Plot band diagram and complete J-V with theoretical comparison
try:
    # Band diagram
    sim_complete.plot_band_diagram(title='Solar Cell Band Diagram')

    # J-V data
    iv_complete = sim_complete.get_iv_data()
    V_comp = iv_complete.get_voltages(1)      # electrode 1 = front/sweep
    I_comp = iv_complete.get_currents(1)
    V_comp = np.array(V_comp)
    J_comp = np.array(I_comp) / A_device_cm2  # normalise to A/cm^2

    # Theoretical curve for the complete simulation parameters
    J0_comp = calc_J0(
        Na=1e16, Nd=1e19, tau_n=1e-5, tau_p=1e-5,
        W_base=200e-4, W_emit=0.5e-4,
        front_srv=1e3, back_srv=1e5
    )
    V_th = np.linspace(0.1, 0.75, 300)
    J_theory = shockley_J(V_th, J0_comp)

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=V_th, y=J_theory, mode='lines',
                   line=dict(color='black', dash='dash', width=2),
                   name=f'Shockley theory (J0={J0_comp:.2e} A/cm²)')
    )
    J_pos = J_comp.copy()
    J_pos[J_pos <= 0] = np.nan
    fig.add_trace(
        go.Scatter(x=V_comp, y=J_pos, mode='lines+markers',
                   line=dict(color='royalblue', width=2),
                   marker=dict(size=4),
                   name='PADRE (normalised)')
    )

    fig.update_xaxes(title_text='Voltage (V)')
    fig.update_yaxes(title_text='J (A/cm²)', type='log')
    fig.update_layout(
        title_text='Complete Dark J-V Characteristic',
        template='plotly_white', width=900, height=450
    )
    fig.show()

    print(f'Theoretical J0 = {J0_comp:.3e} A/cm²')
    print(f'(with front SRV=1e3 cm/s, back SRV=1e5 cm/s)')

except Exception as e:
    print(f'Could not create plots: {e}')
    import traceback; traceback.print_exc()


---

## 8. Exercises

### Exercise 1: P-on-N Solar Cell
Create a P-on-N structure and compare it to the N-on-P design.

In [ ]:
# Exercise 1: P-on-N solar cell — compare with N-on-P
sim_pon = create_solar_cell(
    emitter_depth=0.5,
    base_thickness=200.0,
    emitter_doping=1e19,
    base_doping=1e16,
    device_type='p_on_n',
    temperature=300,
    srh=True,
    taun0=1e-5,
    taup0=1e-5,
    log_iv=True,
    forward_sweep=(0.0, 0.7, 0.02)
)

print('Running P-on-N simulation...')
result_pon = sim_pon.run()

if result_pon.returncode == 0:
    print('Simulation completed!')

    # Theoretical J0 for both structures
    # N-on-P: minority electrons in p-base, minority holes in n-emitter
    J0_nop = calc_J0(Na=1e16, Nd=1e19, tau_n=1e-5, tau_p=1e-5,
                     W_base=200e-4, W_emit=0.5e-4,
                     front_srv=1e4, back_srv=1e7)
    # P-on-N: minority holes in n-base (Na↔Nd swapped)
    J0_pon = calc_J0(Na=1e19, Nd=1e16, tau_n=1e-5, tau_p=1e-5,
                     W_base=200e-4, W_emit=0.5e-4,
                     front_srv=1e4, back_srv=1e7)

    V_th = np.linspace(0.1, 0.75, 300)

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=V_th, y=shockley_J(V_th, J0_nop), mode='lines',
                   line=dict(color='royalblue', dash='dash', width=2),
                   name=f'N-on-P theory (J0={J0_nop:.2e} A/cm²)')
    )
    fig.add_trace(
        go.Scatter(x=V_th, y=shockley_J(V_th, J0_pon), mode='lines',
                   line=dict(color='tomato', dash='dash', width=2),
                   name=f'P-on-N theory (J0={J0_pon:.2e} A/cm²)')
    )

    fig.update_xaxes(title_text='Voltage (V)')
    fig.update_yaxes(title_text='J (A/cm²)', type='log')
    fig.update_layout(
        title_text='N-on-P vs P-on-N: Shockley Theory Comparison',
        template='plotly_white', width=900, height=450
    )
    fig.show()

    print(f'N-on-P: J0 = {J0_nop:.3e} A/cm²')
    print(f'P-on-N: J0 = {J0_pon:.3e} A/cm²')
    print()
    print('The structures have the same J0 here because the doping is symmetric.')
    print('In practice, N-on-P vs P-on-N differ mainly in minority carrier type,')
    print('surface passivation effectiveness, and radiation hardness.')
else:
    print('Simulation failed')


---

## Summary

In this notebook, you learned:

1. **Solar Cell Structure**: N-on-P junction with emitter and base
2. **Running PADRE Simulations**: Using `create_solar_cell()` and `sim.run()`
3. **Dark I-V Analysis**: Understanding recombination from simulation
4. **Lifetime Effects**: Diffusion length and collection efficiency
5. **Surface Recombination**: Importance of passivation
6. **Doping Optimization**: Trade-offs in base doping

**Key Equations:**
- $V_{oc} = \frac{kT}{q}\ln(I_L/I_0 + 1)$
- $L = \sqrt{D\tau}$ (diffusion length)
- For good collection: L > Base thickness

**Next**: [07 - MOS Capacitor](07_MOS_Capacitor.ipynb) - Capacitance-voltage analysis